In [6]:
import re
import pandas as pd
from thefuzz import process
from thefuzz import fuzz

# --- STEP 1: DEFINE THE TOOLS ---

def clean_for_match(text):
    """Strips symbols like ® and ™ to allow exact matching."""
    if not text or pd.isna(text):
        return ""
    text = str(text).lower()
    # Remove symbols and punctuation
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return " ".join(text.split()).strip()

def find_best_matches(unmatched_list, choices_list, threshold=85):
    """Finds siblings for titles that failed exact matching."""
    fuzzy_results = {}
    for title in unmatched_list:
        if not title: continue
        # Fuzzy search against the cleaned Google Sheet list
        match, score = process.extractOne(title, choices_list, scorer=fuzz.token_set_ratio)
        if score >= threshold:
            fuzzy_results[title] = match
    return fuzzy_results

In [7]:
# --- THE TEST DATA ---
# Let's simulate your Bratz and Halo issues
api_titles = ["Bratz® Rhythm & Style", "Halo Infinite (Campaign)", "Gears 5 - Standard"]
gsheet_titles = ["Bratz Rhythm & Style", "Halo Infinite", "Gears 5"]

# 1. Clean them using your regex function
clean_api = [clean_for_match(t) for t in api_titles]
clean_gsheet = [clean_for_match(t) for t in gsheet_titles]

print(f"Cleaned API: {clean_api}")
print(f"Cleaned Sheet: {clean_gsheet}")

# 2. Test the Fuzzy Matcher
matches = find_best_matches(clean_api, clean_gsheet)

print("\n--- RESULTS ---")
for original, found in matches.items():
    print(f"Original Cleaned: '{original}' -> Matched to: '{found}'")

Cleaned API: ['bratz rhythm style', 'halo infinite campaign', 'gears 5 standard']
Cleaned Sheet: ['bratz rhythm style', 'halo infinite', 'gears 5']

--- RESULTS ---
Original Cleaned: 'bratz rhythm style' -> Matched to: 'bratz rhythm style'
Original Cleaned: 'halo infinite campaign' -> Matched to: 'halo infinite'
Original Cleaned: 'gears 5 standard' -> Matched to: 'gears 5'
